In [1]:
from zeta_py.zeta_client.accounts.pricing import Pricing
from solana.rpc.async_api import AsyncClient
from zeta_py.client import Client
from zeta_py.types import Network, Asset, Side
import anchorpy
from solders.pubkey import Pubkey
from zeta_py import pda
from solana.rpc.commitment import Confirmed, Finalized

conn = AsyncClient("https://api.mainnet-beta.solana.com", commitment=Confirmed, blockhash_cache=False)

wallet = anchorpy.Wallet.local()  # get local filesystem keypair wallet
print(wallet.public_key)

4mYxEXiFwe3ePEmoj7CYCbiW7wGTojUQCmdPqGaoVaUh


In [2]:
# print(f"Balance: {(await conn.get_balance(wallet.public_key)).value / 1e9} SOL")

Balance: 0.24391817 SOL


In [2]:
client = await Client.load(Network.MAINNET, conn, wallet, [Asset.SOL])

INFO:zeta_py.accounts.Account:Loaded account: State
INFO:zeta_py.accounts.Account:Loaded account: Pricing
INFO:zeta_py.accounts.Account:Loaded account: Clock
INFO:zeta_py.accounts.Account:Loaded account: CrossMarginAccount


In [9]:
# print(client.balance)
# print(client.positions)
# print(client.open_orders)

0.009825
{<Asset.SOL: 0>: Position(size=0.001, cost_of_trades=0.021262)}
{<Asset.SOL: 0>: [Order(order_id=184467459183839589766535671, client_id=0, open_order_address=Pubkey(
    ciwBQfLdM97vFp8GwYYLczPgYoLfdkTVD6dhHAeGoWj,
), open_order_slot=0, fee_tier=0, info=OrderInfo(price=10.0, size=0.001, price_lots=10000000, size_lots=1), side=<Side.BID: 0>, tif_offset=0)]}


In [3]:
await client.deposit(0.000001)

INFO:zeta_py.accounts.Account:Loaded account: CrossMarginAccountManager
INFO:zeta_py.client.Client:Deposit of $1e-06 USDC to margin account Hu2yXbPESXyamLdvDemeASCFd3rHnpN2xBDUJpnuG5UX submitted


Signature(
    4CBqjwMygd6i8rQUayeDewVCk2ucwWumYkqX7c7gF8yybL6mn9oRhKfDA6Bao6rqyukgE8ei77Ag6gfRN8bT2Fk5,
)

In [6]:
await client.place_order(Asset.SOL, 0.1, 0.001, Side.ASK)

[Order(order_id=1844692854115028764209851, client_id=0, open_order_address=Pubkey(
    ciwBQfLdM97vFp8GwYYLczPgYoLfdkTVD6dhHAeGoWj,
), open_order_slot=1, fee_tier=0, info=OrderInfo(price=0.1, size=0.001, price_lots=100000, size_lots=1), side=<Side.BID: 0>, tif_offset=0), Order(order_id=184467459183839589766535671, client_id=0, open_order_address=Pubkey(
    ciwBQfLdM97vFp8GwYYLczPgYoLfdkTVD6dhHAeGoWj,
), open_order_slot=0, fee_tier=0, info=OrderInfo(price=10.0, size=0.001, price_lots=10000000, size_lots=1), side=<Side.BID: 0>, tif_offset=0)]



INFO:zeta_py.client.Client:Placed order $SOL-PERP ASK $0.001 @ $0.1


Signature(
    462U6rABdP39fg3hQbK753gtwbDFC4yyk6gssUtdfKpsv4yUgeSZ6nHQ1usA8zHsE3SLHjUqiJKwksPu5YTbbMkG,
)

In [8]:
from solana.transaction import Transaction
from zeta_py import constants, utils
from zeta_py.types import OrderOptions
from spl.token.constants import ASSOCIATED_TOKEN_PROGRAM_ID, TOKEN_PROGRAM_ID
from solders.sysvar import RENT


from zeta_py.zeta_client.instructions import (
    deposit_v2,
    initialize_cross_margin_account,
    initialize_cross_margin_account_manager,
    place_perp_order_v3,
)

tx = Transaction()

asset = Asset.SOL
price = 0.1
size=0.001
side = Side.BID
order_opts = OrderOptions

tx.add(
    place_perp_order_v3(
                {
                    "price": utils.convert_decimal_to_fixed_int(price),
                    "size": utils.convert_decimal_to_fixed_lot(size),
                    "side": side.to_program_type(),
                    "order_type": order_opts.order_type.to_program_type(),
                    "client_order_id": order_opts.client_order_id,
                    "tif_offset": None,  # TODO: this later `getTIFOffset`
                    "tag": order_opts.tag,
                    "asset": asset.to_program_type(),
                },
                {
                    "state": client.exchange.state.address,
                    "pricing": client.exchange.pricing.address,
                    "margin_account": client.margin_account.address,
                    "authority": client.provider.wallet.public_key,
                    "dex_program": constants.DEX_PID[client.network],
                    "token_program": TOKEN_PROGRAM_ID,
                    "serum_authority": client.exchange._serum_authority_address,
                    "open_orders": client._open_orders_addresses[asset],
                    "rent": RENT,
                    "market_accounts": {
                        "market": client.exchange.markets[asset].address,
                        "request_queue": client.exchange.markets[asset]._serum_market.state.request_queue(),
                        "event_queue": client.exchange.markets[asset]._serum_market.state.event_queue(),
                        "bids": client.exchange.markets[asset]._serum_market.state.bids(),
                        "asks": client.exchange.markets[asset]._serum_market.state.asks(),
                        "order_payer_token_account": client.exchange.markets[asset]._quote_zeta_vault_address
                        if side == Side.BID
                        else client.exchange.markets[asset]._base_zeta_vault_address,
                        "coin_vault": client.exchange.markets[asset]._serum_market.state.base_vault(),
                        "pc_vault": client.exchange.markets[asset]._serum_market.state.quote_vault(),
                        "coin_wallet": client.exchange.markets[asset]._base_zeta_vault_address,
                        "pc_wallet": client.exchange.markets[asset]._quote_zeta_vault_address,
                    },
                    "oracle": client.exchange.pricing.account.oracles[asset.to_index()],
                    "oracle_backup_feed": client.exchange.pricing.account.oracle_backup_feeds[asset.to_index()],
                    "oracle_backup_program": constants.CHAINLINK_PID,
                    "market_mint": client.exchange.markets[asset]._serum_market.state.quote_mint()
                    if side == Side.BID
                    else client.exchange.markets[asset]._serum_market.state.base_mint(),
                    "mint_authority": client.exchange._mint_authority_address,
                    "perp_sync_queue": client.exchange.pricing.account.perp_sync_queues[asset.to_index()],
                },
            )
)

# TODO: grab recent blockhash??
signed_tx = client.provider.wallet.sign_transaction(tx)
signed_tx.verify_signatures()

await client.provider.simulate(signed_tx)

InvalidParamsMessage {
    message: "invalid transaction: Transaction failed to sanitize accounts offsets correctly",
}